# An overview of classification

## What is classification?

Just as in the regression setting, in the classification setting we have a set of training observations $ (x_1, y_1), \dots, (x_n, y_n) $, where each $ y_i $ is a qualitative response. 

In this chapter, we will illustrate the concept of classification using the simulated `Default` data set. We are interested in predicting whether an individual will default on his or her credit card payment, which takes on the value `Yes` if the customer defaults on their credit card payment and `No` if they do not, on the basis of annual income and monthly credit card balance.

Run the code cells below to load the libraries needed and the `Default` data as a pandas dataframe.

In [ ]:
import pandas as pd
import numpy as np
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression

# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
# from sklearn.metrics import confusion_matrix, classification_report, precision_score
# from sklearn import preprocessing
# from sklearn import neighbors

from statsmodels.formula.api import logit

%matplotlib inline

In [ ]:
data_url = "https://github.com/pykale/transparentML/raw/main/data/Default.csv"
df = pd.read_csv(data_url)

# Note: factorize() returns two objects: a label array and an array with the unique values.
# We are only interested in the first object.
df["default2"] = df.default.factorize()[0]
df["student2"] = df.student.factorize()[0]
df.head(3)

Run the following code to display 10,000 individuals' data of annual `income` and monthly credit card `balance`, and the relationship with the `default` status.

In [ ]:
fig = plt.figure(figsize=(12, 5))
gs = GridSpec(1, 4)
ax1 = plt.subplot(gs[0, :-2])
ax2 = plt.subplot(gs[0, -2])
ax3 = plt.subplot(gs[0, -1])

# Take a fraction of the samples where target value (default) is 'no'
df_no = df[df.default2 == 0].sample(frac=0.15)
# Take all samples  where target value is 'yes'
df_yes = df[df.default2 == 1]
#
df_ = pd.concat((df_no, df_yes))

ax1.scatter(
    df_[df_.default == "Yes"].balance,
    df_[df_.default == "Yes"].income,
    s=40,
    c="orange",
    marker="+",
    linewidths=1,
)
ax1.scatter(
    df_[df_.default == "No"].balance,
    df_[df_.default == "No"].income,
    s=40,
    marker="o",
    linewidths=1,
    edgecolors="lightblue",
    facecolors="white",
    alpha=0.6,
)
#
ax1.set_ylim(ymin=0)
ax1.set_ylabel("Income")
ax1.set_xlim(xmin=-100)
ax1.set_xlabel("Balance")

c_palette = {"No": "lightblue", "Yes": "orange"}
sns.boxplot(data=df, y="balance", x="default", orient="v", ax=ax2, palette=c_palette)
sns.boxplot(data=df, y="income", x="default", orient="v", ax=ax3, palette=c_palette)
gs.tight_layout(plt.gcf())
# plt.show()

```{admonition} Observations from the plot
:class: tip, dropdown
In the left-hand panel of the above figure, the individuals who defaulted in a given month are shown in orange, and those who did not in blue. (The overall default rate is about 3\%, so we have plotted only a fraction of the individuals who did not default.) It appears that individuals who defaulted tended to have higher credit card balances than those who did not. 

In the center and right-hand panels, two pairs of boxplots are shown. The first shows the distribution of balance split by the binary default variable; the second is a similar plot for income. 
<!-- In this chapter, we learn how to build a model to predict default ($y$) for any given value of balance ($x_1$) and income ($x_2$). Since $y$ is not quantitative, the simple linear regression model of Chapter 3 is not a good choice: we will elaborate on this in the next section.  -->
It is worth noting that this example displays a very pronounced relationship between the predictor balance and the response default. In most real applications, the relationship between the predictor and the response will not be nearly so strong. To illustrate the classification procedures discussed, we use this example in which the relationship between the predictor and the response is somewhat exaggerated.
```

## Why not linear regression?

There are two main reasons not to perform classification using a regression method: 

- A regression method cannot accommodate a qualitative response with more than two classes; 
- A regression method will not provide meaningful estimates of $\mathbb{P}(y|x)$

Suppose there are three medical conditions of patients in a emergency room: `stroke`, `drug overdose`, and `epileptic seizure`. We could consider encoding these values as a quantitative response variable, $y$, as follows:

$$
y = \begin{cases} 1 & \text{if stroke}, \\ 2 & \text{if drug overdose}, \\ 3 & \text{if epileptic seizure}. \end{cases}
$$

Using this coding, least squares could be used to fit a linear regression model to predict $y$. However, this coding implies that `drug overdose` is a condition in between `stroke` and `epileptic seizure`, and insisting that the difference between stroke and drug overdose is the same as the difference between drug overdose and epileptic seizure . In practice there is no particular reason that this needs to be the case. For instance, one could choose an equally reasonable coding,

$$
y = \begin{cases} 1 & \text{if epileptic seizure}, \\ 2 & \text{if stroke}, \\ 3 & \text{if drug overdose}, \end{cases}
$$

which would imply a totally different relationship among the three conditions. Each of these codings would produce fundamentally different linear models that would ultimately lead to different sets of predictions on test observations.

If the response variable’s values did take on a natural ordering, such as _mild, moderate, and severe_, and we felt the gap between mild and moderate was similar to the gap between moderate and severe, then a 1, 2, 3 coding would be reasonable. Unfortunately, in general there is no natural way to convert a qualitative response variable with more than two levels into a
quantitative response that is ready for linear regression.

For a binary (two level) qualitative response, the situation is better. For binary instance, perhaps there are only two possibilities for the patient’s medical condition: `stroke` and `drug overdose`. We could then code the response as
follows:

$$
y = \begin{cases} 1 & \text{if stroke}, \\ 0 & \text{if drug overdose}. \end{cases}
$$

We could then fit a linear regression to this binary response, and predict `drug overdose` if $ y > 0.5 $ and `stroke` otherwise. In the binary case it is not hard to show that even if we flip the above coding, linear regression will produce the same final predictions.

Run the following code to compare the difference between the linear regression model and the logistic regression model for a binary classification task on the `Default` dataset.

In [ ]:
X_train = df.balance.values.reshape(-1, 1)
y = df.default2

# Create array of test data. Calculate the classification probability
# and predicted classification.
X_test = np.arange(df.balance.min(), df.balance.max()).reshape(-1, 1)

clf = LogisticRegression(solver="newton-cg")
clf.fit(X_train, y)
prob = clf.predict_proba(X_test)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
# Left plot
sns.regplot(
    x=df.balance,
    y=df.default2,
    order=1,
    ci=None,
    scatter_kws={"color": "orange"},
    line_kws={"color": "lightblue", "lw": 2},
    ax=ax1,
)
# Right plot
ax2.scatter(X_train, y, color="orange")
ax2.plot(X_test, prob[:, 1], color="lightblue")

for ax in fig.axes:
    ax.hlines(
        1,
        xmin=ax.xaxis.get_data_interval()[0],
        xmax=ax.xaxis.get_data_interval()[1],
        linestyles="dashed",
        lw=1,
    )
    ax.hlines(
        0,
        xmin=ax.xaxis.get_data_interval()[0],
        xmax=ax.xaxis.get_data_interval()[1],
        linestyles="dashed",
        lw=1,
    )
    ax.set_ylabel("Probability of default")
    ax.set_xlabel("Balance")
    ax.set_yticks([0, 0.25, 0.5, 0.75, 1.0])
    ax.set_xlim(xmin=-100)

As shown in the output figure, for a binary response with a 0/1 coding, although the linear regression model can also produce the an estimate of $\mathbb{P}(y|x)$, some estimates might be outside the $[0, 1]$ interval making them hard to interpret as probabilities. Since the predicted outcome of linear regression is not a probability, but a linear interpolation between points, there is no meaningful threshold at which you can distinguish one class from the other. A good illustration of this issue has been given on [Stackoverflow](https://stats.stackexchange.com/questions/22381/why-not-approach-classification-through-regression).